In [6]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)

In [7]:
filenm = './previous_application.csv.zip'
previous = pd.read_csv(filenm)

In [8]:
previous.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
previous.dtypes

SK_ID_PREV                       int64
SK_ID_CURR                       int64
NAME_CONTRACT_TYPE              object
AMT_ANNUITY                    float64
AMT_APPLICATION                float64
AMT_CREDIT                     float64
AMT_DOWN_PAYMENT               float64
AMT_GOODS_PRICE                float64
WEEKDAY_APPR_PROCESS_START      object
HOUR_APPR_PROCESS_START          int64
FLAG_LAST_APPL_PER_CONTRACT     object
NFLAG_LAST_APPL_IN_DAY           int64
RATE_DOWN_PAYMENT              float64
RATE_INTEREST_PRIMARY          float64
RATE_INTEREST_PRIVILEGED       float64
NAME_CASH_LOAN_PURPOSE          object
NAME_CONTRACT_STATUS            object
DAYS_DECISION                    int64
NAME_PAYMENT_TYPE               object
CODE_REJECT_REASON              object
NAME_TYPE_SUITE                 object
NAME_CLIENT_TYPE                object
NAME_GOODS_CATEGORY             object
NAME_PORTFOLIO                  object
NAME_PRODUCT_TYPE               object
CHANNEL_TYPE             

In [20]:
AGGS = {
    'SK_ID_PREV': ['nunique','max'] ,
    'AMT_APPLICATION':['sum','mean','max','min',] ,
    'AMT_CREDIT':['sum','mean','max','min',] ,
    'AMT_DOWN_PAYMENT':['sum','mean',] ,
    'AMT_GOODS_PRICE':['sum','mean','max','min',] ,
    'RATE_INTEREST_PRIMARY':['mean',] ,
    'RATE_INTEREST_PRIVILEGED':['mean',] ,
}
prev_aggs = previous.groupby('SK_ID_CURR').agg(AGGS)
prev_aggs.columns = [ 'PREV_' + '_'.join(col).strip() for col in prev_aggs.columns.values]
prev_aggs.reset_index(inplace=True)

# add new calculations 
prev_aggs['prev_pct_down'] = prev_aggs.PREV_AMT_DOWN_PAYMENT_sum / prev_aggs.PREV_AMT_APPLICATION_sum
prev_aggs['prev_pct_credit_app'] = prev_aggs.PREV_AMT_CREDIT_sum / prev_aggs.PREV_AMT_APPLICATION_sum

In [22]:
prev_aggs.replace(np.inf, np.nan, inplace=True)

In [99]:
# get last loan 
last_loan = pd.merge(previous, prev_aggs[['PREV_SK_ID_PREV_max']], left_on=['SK_ID_PREV'],right_on=['PREV_SK_ID_PREV_max'])
last_loan.columns = [ 'lastloan_'+c if c not in ['SK_ID_PREV','SK_ID_CURR'] else c for c in last_loan.columns]

# merge into aggregated dataset
prevs_w_last = pd.merge(prev_aggs, last_loan, how='left', on=['SK_ID_CURR'], suffixes=('','_lastloan'))
del prevs_w_last['PREV_SK_ID_PREV_max']

In [101]:
prevs_w_last.dtypes

SK_ID_CURR                                int64
PREV_AMT_DOWN_PAYMENT_sum               float64
PREV_AMT_DOWN_PAYMENT_mean              float64
PREV_SK_ID_PREV_nunique                   int64
PREV_RATE_INTEREST_PRIMARY_mean         float64
PREV_AMT_CREDIT_sum                     float64
PREV_AMT_CREDIT_mean                    float64
PREV_AMT_CREDIT_max                     float64
PREV_AMT_CREDIT_min                     float64
PREV_AMT_APPLICATION_sum                float64
PREV_AMT_APPLICATION_mean               float64
PREV_AMT_APPLICATION_max                float64
PREV_AMT_APPLICATION_min                float64
PREV_AMT_GOODS_PRICE_sum                float64
PREV_AMT_GOODS_PRICE_mean               float64
PREV_AMT_GOODS_PRICE_max                float64
PREV_AMT_GOODS_PRICE_min                float64
PREV_RATE_INTEREST_PRIVILEGED_mean      float64
prev_pct_down                           float64
prev_pct_credit_app                     float64
SK_ID_PREV                              

In [4]:
def prepare_previous_loans(filenm='./previous_application.csv.zip'):
    AGGS = {
        'SK_ID_PREV': ['nunique','max'] ,
        'AMT_APPLICATION':['sum','mean','max','min',] ,
        'AMT_CREDIT':['sum','mean','max','min',] ,
        'AMT_DOWN_PAYMENT':['sum','mean',] ,
        'AMT_GOODS_PRICE':['sum','mean','max','min',] ,
        'RATE_INTEREST_PRIMARY':['mean',] ,
        'RATE_INTEREST_PRIVILEGED':['mean',] ,
    }
    previous = pd.read_csv(filenm)
    prev_aggs = previous.groupby('SK_ID_CURR').agg(AGGS)
    prev_aggs.columns = [ 'PREV_' + '_'.join(col).strip() for col in prev_aggs.columns.values]
    prev_aggs.reset_index(inplace=True)

    # add new calculations 
    prev_aggs['prev_pct_down'] = prev_aggs.PREV_AMT_DOWN_PAYMENT_sum / prev_aggs.PREV_AMT_APPLICATION_sum
    prev_aggs['prev_pct_credit_app'] = prev_aggs.PREV_AMT_CREDIT_sum / prev_aggs.PREV_AMT_APPLICATION_sum
    prev_aggs.replace(np.inf, np.nan, inplace=True)    

    # get last loan 
    last_loan = pd.merge(previous, prev_aggs[['PREV_SK_ID_PREV_max']], left_on=['SK_ID_PREV'],right_on=['PREV_SK_ID_PREV_max'])
    last_loan.columns = [ 'lastloan_'+c if c not in ['SK_ID_PREV','SK_ID_CURR'] else c for c in last_loan.columns]

    # merge into aggregated dataset
    prevs_w_last = pd.merge(prev_aggs, last_loan, how='left', on=['SK_ID_CURR'], suffixes=('','_lastloan'))
    del prevs_w_last['PREV_SK_ID_PREV_max']
    
    return prevs_w_last